In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv, dotenv_values
load_dotenv()


True

In [6]:
import requests

headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": "GITHUB_TOKEN"
}

query = "machine learning"
url = f"https://api.github.com/search/repositories?q={query}&sort=stars&order=desc&per_page=1&page=1"

response = requests.get(url, headers=headers)
data = response.json()

if response.status_code != 200:
    print("Error:", data.get("message", "Unknown error"))
    print("Status code:", response.status_code)
    print("Response headers:", response.headers)
else:
    repo = data["items"][0]
    print("Top repo:")
    print("Name:", repo["full_name"])
    print("URL:", repo["html_url"])


Error: API rate limit exceeded for 8d9f7747675e24454cd9b7ed35c58707:Repositories. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.) If you reach out to GitHub Support for help, please include the request ID E508:2F073E:3F421AB:414B8FE:67E45C71 and timestamp 2025-03-26 19:58:42 UTC.
Status code: 403
Response headers: {'Date': 'Wed, 26 Mar 2025 19:58:42 GMT', 'Content-Type': 'application/json; charset=utf-8', 'X-RateLimit-Limit': '10', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1743019144', 'X-RateLimit-Used': '135', 'X-RateLimit-Resource': 'search', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Suns

In [ ]:

add to a folder
get requirements, dockerfile, 


In [18]:
# https://chatgpt.com/c/67e2fa1a-2850-8008-99a8-bd908a6eb335


In [2]:
# 🔍 1. Scrape and Analyze Public Repositories (e.g., GitHub)
# What to look for:

# ML frameworks: TensorFlow, PyTorch, Hugging Face Transformers, etc.

# Infrastructure: Docker, Kubernetes, Terraform

# Languages: Python, Java, Go, etc.

# Tooling: MLflow, Weights & Biases, Ray, etc.

# How to do it:

# Use GitHub’s API to pull repo metadata, requirements.txt, Dockerfile, etc.

# Parse for keywords and dependencies.

# Analyze commit history and contributors to see which teams or engineers worked on what.

SyntaxError: invalid character '🔍' (U+1F50D) (3781455242.py, line 1)

In [ ]:
# 🧠 2. Use Job Postings for Stack Clues
# What to look for:

# Job listings for ML Engineers, MLOps, or Applied Scientists.

# Stacks and tools mentioned (they’re usually very specific).

# Tools to help:

# Diffbot or Apollo.io for scraping and structuring job post data.

# Simple web scraping + LLM pipeline to extract and summarize

In [ ]:
# 🌐 3. Explore Company Engineering Blogs & Tech Talks
# Places to check:

# Medium, Dev.to, Substack, company tech blogs.

# YouTube talks or conference slides (e.g., NeurIPS, PyData, MLConf).

# How to analyze:

# Use an agent (e.g. LangChain + LLM) to summarize or cross-reference stacks across companies.

In [ ]:
# 🧩 4. LinkedIn Engineering Profiles
# Use case:

# Search for engineers in ML roles at the target companies.

# Analyze their LinkedIn summaries and endorsements for tooling or stack mentions.

# Bonus: You can build a mini-graph of tools used across the org based on who lists what.

In [ ]:
# 📦 5. Inspect Their Products (Client/Server Side)
# For web-based ML products:

# Use browser dev tools or tools like Wappalyzer to detect front-end and API tools.

# Look for WebAssembly, ONNX.js, TensorFlow.js, etc.

# For APIs:

# If their ML product exposes APIs, use Postman or curl to inspect endpoints and infer architecture.

In [ ]:
# 🧪 6. Use Prebuilt Tools & Databases
# Examples:

# StackShare.io – Check stack disclosures by company.

# Crunchbase – Look at ML acquisitions, product descriptions.

# BuiltWith – Web tech stack detection.

# Papers With Code – For more research-y orgs, see what papers are tied to which models and codebases.

In [ ]:
# 🧠 7. Build an Agentic AI Researcher
# If you're aiming for scale, consider building an agent that:

# Takes in a list of companies.

# Searches GitHub, job sites, blogs, LinkedIn.

# Extracts stack mentions.

# Aggregates and ranks the tools based on relevance or frequency.